<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #10: Twitter
`Fecha de entrega: Noviembre 14, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [4]:
# cargar librerias 

# Estos paquetes los usaremos para analizar y visualizar los resultados
import pandas as pd
import tweepy
import re
import regex

# Libreias Plotly
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

from collections import Counter

import urllib.parse
import urllib.request
import time
import hmac
import hashlib
import random
import base64
import json

# Este es un modulo local de Python que contiene las credenciales de acceso para Twitter
import tweepy


In [5]:
# Leer llaves
keys = pd.read_csv("C:/Users/practicas/Desktop/Claves Tuit.csv", header=None)
keys= dict(zip(keys[0],keys[1]))

In [6]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'],keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True) # se autoregula para recolectarinformacion

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

(Si exponen sus claves, les haré maldades 😈)

❌🎃❌ ¡No Halloween!

In [7]:
# Cargar archivo ciudades lat	ln bogota
ciudades = pd.read_csv("C:/Users/practicas/Documents/co_small.csv")
ciudades.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Bogotá,4.649178,-74.062827,Colombia,CO,Bogotá,primary,7772000.0,6333661.0
1,Medellín,6.251840,-75.563591,Colombia,CO,Antioquia,admin,3297000.0,1999979.0
2,Cali,3.437222,-76.522500,Colombia,CO,Valle del Cauca,admin,2254000.0,2178836.0
3,Barranquilla,10.963889,-74.796389,Colombia,CO,Atlántico,admin,1798000.0,1244491.0
4,Bucaramanga,7.125393,-73.119804,Colombia,CO,Santander,admin,1009000.0,571820.0


In [8]:
tweets = tweepy.Cursor(api.search,
                       q=['Seleccion colombia 2020 -RT'], # RT no incluir los retuis
                       lang='es', # solo tui en español
                           geocode=f'4.649178,-74.062827,45mi', #direccion geografica  bogota cambia a la que se quiera
                       tweet_mode='extended', # para que no recorte los texto del tuit
                       include_rts=False).items(100) # limitar el numero de twitwes

In [9]:
t = []


for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogotá")
    temp.append(tweet.user.location)
    temp.append(tweet.place)
    temp.append(tweet.coordinates)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)


In [10]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'place', 'coordinates', 'id', 'link']
data

,tweet,date,city,user_location,place,coordinates,id,link
0,"A propósito del fútbol... \n""Selección Colombi...",2020-11-11 22:30:42,Bogotá,"Bogotá, Colombia",None,None,1326653605093519367,https://twitter.com/i/web/status/1326653605093...
1,#SelecciónColombia\n\nEl lateral derecho quedó...,2020-11-11 14:01:36,Bogotá,Bogotá,None,None,1326525485585739777,https://twitter.com/i/web/status/1326525485585...
2,David Ospina podría ser baja de la Selección C...,2020-11-10 23:52:17,Bogotá,"Colombia, Bogotá",None,None,1326311749956497408,https://twitter.com/i/web/status/1326311749956...
3,Procesos de #Reclutamiento y Selección Inclusi...,2020-11-10 15:36:00,Bogotá,Bogotá,None,None,1326186854991994880,https://twitter.com/i/web/status/1326186854991...
4,Álvaro Montero del #DeportesTolima /D|T\ fue e...,2020-11-10 02:22:08,Bogotá,Ibagué - Tolima,None,None,1325987073267036160,https://twitter.com/i/web/status/1325987073267...
5,Luis Suárez y otras 7 variantes hizo Carlos Qu...,2020-11-09 21:40:03,Bogotá,"Bogotá, Colombia",None,None,1325916085959462913,https://twitter.com/i/web/status/1325916085959...
6,#SelecciónColombia\n#VamosColombia\n\nEl entre...,2020-11-09 21:00:22,Bogotá,Bogotá,None,None,1325906097878806529,https://twitter.com/i/web/status/1325906097878...
7,Procesos de #Reclutamiento y Selección Inclusi...,2020-11-09 15:36:00,Bogotá,Bogotá,None,None,1325824467092688898,https://twitter.com/i/web/status/1325824467092...
8,#Deporte | Del 10 al 15 de noviembre cerca de ...,2020-11-09 13:05:51,Bogotá,"Ibague, Tolima",None,None,1325786682894012416,https://twitter.com/i/web/status/1325786682894...
9,#DeporteReactiva | Cerca de 200 deportistas de...,2020-11-09 12:26:33,Bogotá,"Ibagué, Colombia",None,None,1325776790992908288,https://twitter.com/i/web/status/1325776790992...


# Pre_ Prosesamiento twiters

In [11]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,"A propósito del fútbol... \n""Selección Colombi...",2020-11-11 22:30:42,Bogotá
1,#SelecciónColombia\n\nEl lateral derecho quedó...,2020-11-11 14:01:36,Bogotá
2,David Ospina podría ser baja de la Selección C...,2020-11-10 23:52:17,Bogotá
3,Procesos de #Reclutamiento y Selección Inclusi...,2020-11-10 15:36:00,Bogotá
4,Álvaro Montero del #DeportesTolima /D|T\ fue e...,2020-11-10 02:22:08,Bogotá


# Preprosesamiento
- quitar enlaces y menciones
- quitar emojis

In [12]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

In [13]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [14]:
# nueva columna
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,tweet,date,city,clean,links,hashtags,mentions
0,"A propósito del fútbol... \n""Selección Colombi...",2020-11-11 22:30:42,Bogotá,"A propósito del fútbol... \n""Selección Colombi...",[https://t.co/4Hgs1pcoFT],[],[]
1,#SelecciónColombia\n\nEl lateral derecho quedó...,2020-11-11 14:01:36,Bogotá,\n\nEl lateral derecho quedó huérfano sin Aria...,[https://t.co/X6xneFO76Z],[#SelecciónColombia],[]
2,David Ospina podría ser baja de la Selección C...,2020-11-10 23:52:17,Bogotá,David Ospina podría ser baja de la Selección C...,[https://t.co/PL1KeoRzKN],[],[]
3,Procesos de #Reclutamiento y Selección Inclusi...,2020-11-10 15:36:00,Bogotá,Procesos de y Selección Inclusivos - Curso Vi...,"[https://t.co/cHGijAdjIv, https://t.co/y2clqrb...","[#Reclutamiento, #InclusiónLaboral;]",[]
4,Álvaro Montero del #DeportesTolima /D|T\ fue e...,2020-11-10 02:22:08,Bogotá,Álvaro Montero del /D|T\ fue el primero en un...,"[https://t.co/u50FccGR9U, https://t.co/vKunJG5...","[#DeportesTolima, #Colombia, #SantaFe]",[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

❌🎃❌ ¡No Halloween!

In [15]:
# contar por dias cuantos tuis hubieron 
data['fecha'] = data.date.dt.date 
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
8,2020-11-03,1
7,2020-11-04,1
6,2020-11-05,1
1,2020-11-06,6
2,2020-11-07,4
3,2020-11-08,4
0,2020-11-09,8
4,2020-11-10,3
5,2020-11-11,2


In [16]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})


layout = go.Layout(title= 'Selección Colombia 2020  ...tuits por día')

 
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)